<img src="resources/physicsOtago.png" width="600">

$$
\def\julia{\texttt{julia}}
\def\dashint{{\int\!\!\!\!\!\!-\,}}
\def\infdashint{\dashint_{\!\!\!-\infty}^{\,\infty}}
\def\D{\,{\rm d}}
\def\E{{\rm e}}
\def\dx{\D x}
\def\dt{\D t}
\def\dz{\D z}
\def\C{{\mathbb C}}
\def\R{{\mathbb R}}
\def\CC{{\cal C}}
\def\HH{{\cal H}}
\def\I{{\rm i}}
\def\Res_#1{\underset{#1}{\rm Res}}\,
\def\sech{{\rm sech}\,}
\def\vc#1{{\mathbf #1}}
\def\qq{\qquad\qquad}
\def\qfor{\qquad\hbox{for}\qquad}
\def\qwhere{\qquad\hbox{where}\qquad}
\def\ale#1{\begin{align}#1\end{align}}
\def\em{\epsilon_m}
\def\unit#1{{\rm #1}}
$$

# PHSI 365: Computational Physics <img align="right" src="resources/3dquenchslab.gif" width="500" height="500">

University of Otago 2020

Ashton Bradley (Coordinator)
<br>
ashton.bradley@otago.ac.nz
<br>

_Image made with_ [Makie.jl](https://github.com/JuliaPlots/Makie.jl)

# Introduction to Julia II

## Finding information
- If you want more than the built-in documentation search, try using google to search the entire website. E.g. Try entering the following search into google `site:https://docs.julialang.org Array` and see what it generates.
- I won't be using [Juno](https://junolab.org/) in this course, but it is installed on the lab machines, and you are welcome to use it if you prefer a gui (you will need to submit notebooks for grading though). To launch, open a terminal, and type `atom` [enter] (built on github's nice text editor [atom](https://www.youtube.com/watch?v=Y7aEiVwBAdk&feature=youtu.be). It has a few advantages if you are writing a lot of code and don't want an integrated scientific document. In particular, it has a nice [fuzzy finder](https://en.wikipedia.org/wiki/Approximate_string_matching) which will do a pretty complete search of all documentation loaded into the current julia session.


# Numbers, types, functions

## e-notation
Some essential input shorthand should be established. We very often need to enter large numbers, for example:

In [1]:
x = 3141000000

3141000000

A much more convenient (and easier to error-check) way to do this is

In [2]:
y = 3.141e9  

3.141e9

However, there is a crucial difference between these two numbers:

In [3]:
typeof(y),typeof(x)

(Float64, Int64)

they are fundamentaly different data types: in __e-notation__ we specify `Float64` numbers, rather than integers.

Operations, e.g. division, will still yield a well-defined result due to type conversion

In [4]:
y/x 

1.0

The convention is that the numbers after `e` are the power of 10 for the number, in `Float64`, so this is a shorthand for 

$$3141\times 10^{6}$$

In [5]:
#If we instead say
z = 3.141f6

3.141f6

In [6]:
typeof(z)

Float32

so that __f-notation__ allows input of `Float32` (single-precision) numbers in an analogous manner (there could be others, but `d` and `g` aren't special in this way). We will say more about numerical accuracy and number representations in the next lecture.

## Range objects
Julia makes use of special kind of object for doing __efficient iteration__ over collections, called a range object. Let's see how this works:

In [7]:
myrange = 1:2:10000

1:2:9999

just a shorthand for the sequence of numbers `startvalue:stepsize:stopvalue`.

In [11]:
typeof(myrange)

StepRange{Int64,Int64}

In [12]:
supertype(typeof(myrange))

OrdinalRange{Int64,Int64}

In [13]:
typeof(myrange)<:OrdinalRange 

true

The `<:` command asks if the left type __is a subtype of__ the right type. In this case, `OrdinalRange` is the supertype of these range objects, so the result is `true`. 

Types form a logical hierarchy in `julia` that allows function behaviour to be tied to the specific types af data that are being passed to the function (this is "optionally typed"). 

## Loops 
There are a number of very nice functions that help with iteration-like commands, in particular in the context of `for` and `while` loops. 

Two of the most useful are `eachindex()`, and `enumerate()`. Let's see how these work in loops.

We have seen how to write basic `for` loops in Lab1. Let's review with some examples.

In [14]:
# Find the sum of the first N integers
N = 99998
s = 0
for i in 1:N # note the use of "in" instead of = here
    s += i
end
s

4999850001

Check against the analytical result (Gauss)

In [15]:
N*(N+1)/2

4.999850001e9

In [16]:
Int(ans) # convert a Float64 to an Int64 if possible (i.e. if it has no non-integer part)

4999850001

We can summarize this as a little function which ensures the ouput is an integer:

In [17]:
sumgauss(N) = N*(N+1)/2 |> Int

sumgauss (generic function with 1 method)

In [18]:
sumgauss(N)

4999850001

Let's say we have a time series of data `x`, sampled at 1000 times:

In [5]:
N=1000
x=randn(N) # samples from the normal distribution, mean zero, variance one. 

1000-element Array{Float64,1}:
  0.5430432929100125 
  1.524691929336536  
 -1.5039374519925897 
 -0.7911642229803135 
 -0.6421318337169633 
  0.689762773659667  
  2.744216750301623  
 -0.17122455706291162
 -1.1484926838159804 
 -0.22093839973652635
  0.3508300154293322 
 -0.23117641374960174
 -0.6189061110191032 
  ⋮                  
  1.1024606381869173 
  2.096279536339154  
 -0.19757039382050082
 -0.29532454947436504
  0.4935272294319918 
 -0.5653805680451759 
 -1.2958353031435432 
  0.8081697704009061 
  1.2185061918783378 
  1.5901313888190558 
 -1.2449098266761243 
 -0.14053971367962315

We can step through and compute something from the series using a for loop:

In [8]:
y = zero(x); # initialize array of zeros of the same type

In [9]:
# Square each element by loop through the vector
for i in 1:N
    y[i] = x[i]^2
end
y

1000-element Array{Float64,1}:
 0.2948960179745497  
 2.324685479383968   
 2.261827859505963   
 0.6259408277240432  
 0.41233329187270973 
 0.475772683926677   
 7.530725572635999   
 0.02931784894139028 
 1.3190354447788335  
 0.0488137764781371  
 0.12308169972614545 
 0.053442534274127054
 0.3830447742567905  
 ⋮                   
 1.215419458751505   
 4.394387894474298   
 0.039034060514387785
 0.08721658952223668 
 0.24356912619081786 
 0.31965518672308574 
 1.6791891328731186  
 0.6531383777898533  
 1.4847573396458484  
 2.5285178337076193  
 1.5498004765547777  
 0.019751411121150454

Another way to achieve the same result:

In [10]:
z = zero(x) # allocates memory, but doesn't set it to anything meaningful
for i in eachindex(x)
    z[i] = x[i].^2 # we can then write to z
end

Let's do a simple check of equivalence of our two results:

In [11]:
z==y # are the values in the two arrays are the same?

true

In [12]:
z===y # are the data and physical addresses in memory the same?

false

Another way to achieve the same thing that can be quite useful is `enumerate`, which gives us both a counter (iterator), and the value for the given value of counter, so we can work with inside the loop:

In [14]:
for (i,xi) in enumerate(x) # declaring name of counter and variable it counts
    z[i] = xi^2 # x is the i'th element of x
end
z==y

true

- Note that we didn't have to say how big `x` is, so the code is less "brittle". 
- These generic commands like `zero` and `eachindex` and `enumerate` can make working with loops a bit more robust and general, and also help the transition from specific code to general functions. 

# Functions II
More typically, our function is too large to fit in one line and we want to define a block of code to take some sensible inputs, do something useful, and return an output. It is important to be familiar with some basic scoping rules in `julia`. This should also help clarify why we want to write (good) functions whenever possible.

There are two main levels of scope for variables in julia:
1. Global (available everywhere, can lead to slow code)
2. Local (can be nested)

__The interior of function falls into the second category.__

Sounds simple on the face of it, but there are a couple of subtleties.

We need to be aware of two rules in julia:

<div class="alert alert-block alert-info">
1. Privacy: Declared variables (named in the function header, or defined within the function) are private to the function
</div>

<div class="alert alert-block alert-info">
2. Context: The function has access to all external variables that sit at the same level of scope as the function declaration; functions can make use of the context in which they are called.
</div>

The corollary is, every variable that you declare as normal in a julia session is __global__. In other words, code run in the REPL, or a jupyter cell (outside a function body) is in __global scope__. 

Let's see how this works with a simple example:


In [29]:
x=2
function g(y)
    return x+y
end

g (generic function with 1 method)

- Here `y` is private, and `x` is available inside `g(y)`: we declared the function at the same scope as the definition of `x`.

- The code will run even thought we aren't explicitly passing `x`:

In [30]:
g(3)

5

- The function `g(y)` responds to changes in `x`:

In [31]:
x=10;g(3)

13

While less "safe" than some other languages, this allows functions to be more focused on a specific task associated with particular inputs, i.e, in the context of solving a problem

> *you don't have to pass half your workspace to get something done!*

It also allows functions to also have context-specific behavior.

Whether this is a good or bad thing may depend on your personal standpoint on langauge design. 

We should be clear here that this discussion is mostly for the sake of completeness: in practice you will most likely not encounter the problem, but it is important to be aware of how functions work in `julia`. Usually people take care to make all of their variables private to a function, but just be aware that it is not a _requirement_ (it is strictly enforced for package code, which is why developers care a lot less about it...). 

## Type stability
A central concept in `julia` is the notion of ___type stability___:

<div class="alert alert-block alert-info">
A function is said to be type stable if it is possible to infer the data type of the outputs by inspection of the data types of the inputs, and the code inside the function.
</div>

In practice the compiler is pretty good at doing this inspection, provided we obey some simple rules when writing our code.

We have seen that there are many convenient methods for type conversion
- [Float32](https://docs.julialang.org/en/v1/base/numbers/#Core.Float32-Tuple{Any}), [Float64](https://docs.julialang.org/en/v1/base/numbers/#Core.Float64-Tuple{Any}), [big](https://docs.julialang.org/en/v1/base/numbers/#Base.big), [complex](https://docs.julialang.org/en/v1/base/numbers/#Base.complex-Tuple{Complex}), etc
- these all support dot-calls
- `.|>` is pretty useful, as are
- [zero](https://docs.julialang.org/en/v1/base/numbers/#Base.zero), [one](https://docs.julialang.org/en/v1/base/numbers/#General-Number-Functions-and-Constants-1), [similar](https://docs.julialang.org/en/v1/base/arrays/#Base.similar)

The latter are examples of methods that help to ensure type stability: by initializing the right data types (e.g. `zero(x)`), your code can remain type-stable throughout, ensuring the most efficient julia output. We will see an example that uses this shortly.

## Geometric series
Let's make a little function to evaluate the geometric series up to a given polynomial degree $N$.

In [15]:
"""
    geomSum(x,N)

Numerically sum `N` terms of the geometric series.
"""
function geomSum(x,N)
    y = 0 # ??? is this a good choice ???
    for k = 0:N-1
        y += x^k
    end
    return y
end

geomSum

In [16]:
?geomSum

search: geomSum get_zero_subnormals



```
geomSum(x,N)
```

Numerically sum `N` terms of the geometric series.


and compare with the analytical result:

<div class="alert alert-block alert-warning"><font color=blue>
    
$$S_N=\sum_{k=0}^{N-1}x^k=1+x+x^2+x^3\dots+x^{N-1}=\frac{1-x^N}{1-x}$$
</font></div>

In [17]:
# here @doc raw lets you write plain latex; without it, you need to do e.g. \\sum
@doc raw"""
    geomExact(x,N)

Analytically sum the first `N` terms of the geometric series

$$S_N=\sum_{k=0}^{N-1}x^k=\frac{1-x^N}{1-x}.$$
"""
geomExact(x,N)=(1-x^N)/(1-x)

geomExact

In [18]:
?geomExact

search: geomExact



```
geomExact(x,N)
```

Analytically sum the first `N` terms of the geometric series

$$
S_N=\sum_{k=0}^{N-1}x^k=\frac{1-x^N}{1-x}.
$$


Our two functions should achieve the same result, but what do we mean by that?

In [19]:
x=0.97
N=100
geomSum(x,N),geomExact(x,N)

(31.74824973581979, 31.748249735819783)

__Codequiz: what am I doing here?__

In [20]:
using LinearAlgebra
≈(a,b) = norm(a-b)<1e-14  

≈ (generic function with 1 method)

In [21]:
geomSum(x,N) ≈ geomExact(x,N)

true

and we can note a small arithmetic error in the last few digits. 

## Dot-calls, broadcasting
So far, we have only defined our function for scalar inputs. At least, that is what we intended to do. In `julia`, it is often very straightforward to evaluate a scalar funtion for arrays of numbers.

Let's redefine our function, this time using `zero(x)` to initialize the geometric series 

In [22]:
@doc raw"""
    geomSum(x,N)

Numerically sum the first `N` terms of the geometric series

$$S_N=\sum_{k=0}^{N-1}x^k=\frac{1-x^N}{1-x}.$$
"""
function geomSum(x,N)
    y = zero(x) # phew, that's better - function is now type stable! 
    for k in 0:N-1
        y += x^k
    end
    return y
end

geomSum

__Codequiz: How do we know if a function is type stable?__

In [23]:
@code_warntype geomSum(.1,3) # we can call it for specific inputs, with a nice macro

Variables
  #self#::Core.Compiler.Const(geomSum, false)
  x::Float64
  N::Int64
  y::Float64
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64

Body::Float64
1 ─       (y = Main.zero(x))
│   %2  = (N - 1)::Int64
│   %3  = (0:%2)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(0, false), Int64])
│         (@_5 = Base.iterate(%3))
│   %5  = (@_5 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_5::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (k = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = y::Float64
│   %12 = (x ^ k)::Float64
│         (y = %11 + %12)
│         (@_5 = Base.iterate(%3, %10))
│   %15 = (@_5 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return y


let's test convergence of our series:

In [25]:
Nvals = [1 5 10 20 30]
xvals = [0.1 0.2 0.3 0.6 0.9]' # transpose

5×1 Adjoint{Float64,Array{Float64,2}}:
 0.1
 0.2
 0.3
 0.6
 0.9

In [26]:
geomSum(xvals,Nvals)

MethodError: MethodError: no method matching -(::Array{Int64,2}, ::Int64)
Closest candidates are:
  -(!Matched::Complex{Bool}, ::Real) at complex.jl:303
  -(!Matched::Missing, ::Number) at missing.jl:115
  -(!Matched::Base.CoreLogging.LogLevel, ::Integer) at logging.jl:107
  ...

In [27]:
geomSum.(xvals,Nvals) # dot-call and broadcasting: the transpose was essential!

5×5 Array{Float64,2}:
 1.0  1.1111  1.11111  1.11111  1.11111
 1.0  1.2496  1.25     1.25     1.25   
 1.0  1.4251  1.42856  1.42857  1.42857
 1.0  2.3056  2.48488  2.49991  2.5    
 1.0  4.0951  6.51322  8.78423  9.57609

- Write (any) scalar function and get an array function for free!
- Over any set of parameters we choose!

## Function names are variables
We can also use a **while loop** with our finite series to evaluate the infinite series to desired numerical precision and test against the analytical result.

In [30]:
"""
    geomInf(x)

Sum up the geometric series, accurate to ~1e-16
"""
function geomInf(x,finiteSeries=geomSum,ϵ=1e-16)
    N = 1
    Sn = geomSum(x,N)
    ΔS = abs(geomSum(x,N+1)-Sn)/Sn # compare sum to N with sum to N+1
    while ΔS > ϵ
        N += 1
        Sn = geomSum(x,N)
        ΔS = abs(geomSum(x,N+1)-Sn)/Sn
    end
    return Sn, N
end

geomInf

In [31]:
x = 0.97
geomInf(x)

(33.3333333333332, 1093)

Let's go for one level of abstraction: since function names are variables, we can make the replacement with any finite sum by introducing a new variable

`geomSum` -> `finiteSum`

We will do this be defining a default input, allowing us to first test our function, and then allow it to accept other finite sums to be evaluated to high precision.

In [42]:
"""
    sumInf(x,finiteSum=geomSum,ϵ=1e-16)

Evaluate the series `finiteSum` to precision given by `ϵ`.  
"""
function sumInf(x,finiteSum=geomSum,ϵ=1e-16)
    N = 1
    Sn = finiteSum(x,N) 
    ΔS = abs(finiteSum(x,N+1)-Sn)/Sn
    while ΔS > ϵ
        N += 1
        Sn = finiteSum(x,N)
        ΔS = abs(finiteSum(x,N+1)-Sn)/Sn
    end
    return Sn, N
end

sumInf

In [43]:
x = 0.97
geomSum(x,2)

1.97

In [44]:
geomSum(x,15)

12.224960362107018

Infinite geometric series: the parital sums $S_N$ converge to 

<div class="alert alert-block alert-warning"><font color=blue>
    
$$S\equiv\sum_{k=0}^{\infty}x^k=\frac{1}{1-x},\quad \text{provided}\quad |x|<1$$
</div></font>

In [45]:
geomA(x)=1/(1-x) # N→∞ limit

geomA (generic function with 1 method)

In [46]:
x = 0.97
sumInf(x)

(33.3333333333332, 1093)

In [47]:
geomA(x)

33.33333333333331

In [48]:
x = .9 # smaller x should require less terms
y1 = sumInf(x)

(9.999999999999993, 329)

In [49]:
y2 = geomA(x)

10.000000000000002

# Flow control and relationals
To finish, we will examine some common logic operations such as AND, and OR.

We have already met methods for determining equivalence of variables: `==` (value) and `===` (both value and physical memory). 

## Ternary operator
Let's use the[ternary operator]() to define a Kronecker-$\delta$

In [59]:
δ(i,j) = i==j ? 1 : 0

δ (generic function with 1 method)

The ternary operator says `if (condition true)? (do this) : (else do this)`

- It turns out we can apply this to many kinds of inputs. Why?
- Can you see how to make the array `B` from above by broadcasting?

In [60]:
δ(0,1),δ(0.,1.),δ(1,1),δ(1.,1.)

(0, 0, 1, 1)

In [61]:
δ("a","a"),δ("a","b")

(1, 0)

In [62]:
δ(3,"icecream") 

0

In [64]:
δ("icecream","icecream") 

1

___Exercise:___ 1. Redirect (`|>`) the output of the above to `typeof`; is the method is type stable? 2. Change the output to be logical (`true`, `false`). 3. Change the output to be `Float64`.

We can even get sensible answers to questions about the equivalence of different types of data.

In [ ]:
x=[0,0,1,0,0]

In [ ]:
y=x

In [ ]:
δ(x,y)

## Equivalence and variable assignment

When are two variables equivalent in julia?

In [51]:
y = randn(100)
a = y; # assigns label "a" to variable y (a now points to y)
b = copy(y); # makes a new variable b in memory

In [52]:
a===y # tells us if the variable is identical physically a==y,a===y

true

In [53]:
a==y,a===y

(true, true)

In [54]:
a==b,a===b

(true, false)

In [55]:
y[2]

0.6274315830976688

In [56]:
a[2]=2.0; y[2] #careful here!

2.0

In [ ]:
a===y #still the same since they point to the same memory! 

> `a` and `y` are both just pointers to the same memory address. If I change `a` I also change `y`. This is not true for `b`.

In [ ]:
b[2]=pi; y[2]

## More examples

In [ ]:
x=rand(20)

In [ ]:
y = similar(x) #create an empty array of the same type (Float64)
for i in eachindex(x)
    y[i]=x[i]^2
end
y

In [ ]:
z = similar(x) 
for i in 1:length(z)
    x[i]>.3 ? z[i]=0.0 : z[i]=x[i]
end
z

In [ ]:
# done as a one-liner via a dot-call:
@. z = (x<0.3)*x

**Bottom line:** logic and relational operators can be applied locally to each element of an array via "dot-calls"

**Note:** logical truth is not equivalent to digital `1 or 0` as in Matlab (but you are free to adjust this as you please).

## Logic
There are many convenient methods for handling logic, in turn allowing us to write more complex code that does different calculations depending on the value of particular inputs. 

In [65]:
typeof(true)

Bool

In [66]:
#~ is logical not (also can use !)
~true,~false, ~~true

(false, true, true)

In [67]:
# == tests for logical equivalence
true==true,true==false, true==~false

(true, false, true)

The simplest way to evaluate logical `AND` is to multiply boolean values:

In [68]:
true*false #true AND false is false

false

In [69]:
true*true #true AND true is true

true

In [70]:
false*false #false AND false is false

false

This often means extra work, if our expressions for true and false come from testing a complicated expression.

## Short-circuit operators
A more efficient binary operator for logical `AND` is 
```julia
A && B
```
means `A AND B`. Thus it only evaluates the expression `B` if `A` is false. Hence it acts as a short circuit for evaluation, avoiding unnecessary computation.

In [75]:
milk = true; honey = true
milk*honey

true

In [76]:
milk && honey

true

Local `A OR B` also has a short-circuit operator

```julia
A || B
```


In [77]:
milk||honey

true

In [78]:
!milk||honey

true

## Truth tables

In [79]:
#truth-table for booleans using array comprehension 
bools=(true,false)

#make a table using array comprehension
println("Truth table for logical AND:")
ANDtruth = [i && j for i in bools, j in bools]

Truth table for logical AND:


2×2 Array{Bool,2}:
 1  0
 0  0

In [80]:
println("Truth table for logical OR:")
ORtruth=[i || j for i in bools, j in bools]

Truth table for logical OR:


2×2 Array{Bool,2}:
 1  1
 1  0

In [ ]:
ANDtruth.*ORtruth  #logical AND locally as a dot-call

# Next time
- Numerical representations and error
    - Accuracy versus precision
    - Roundoff error
    - Underflow, overflow
- Dimensional analysis
    - Dealing with large and small quantities in numerical modelling
    - examples...